# based on the math example in: https://github.com/langchain-ai/langgraph-codeact/blob/harrison/some-changes/examples/math_example.py

In [ ]:
import ollama
import builtins
import contextlib
import io
from typing import Any
import re
import textwrap  # Ensure this is added to your imports
import traceback  # Ensure this is imported at the top
from langchain.chat_models import init_chat_model
from langgraph_codeact import create_codeact


def extract_python_code(response_text: str) -> str:
    """
    Extracts and returns the Python code contained in a markdown code block from the response text.
    Searches for a block delimited by '```python' and '```'. 
    Raises a ValueError if no valid Python code block is found.
    """
    match = re.search(r"```python\n(.*?)\n```", response_text, re.DOTALL)
    if match:
        # Return only the code inside the markdown code block, stripped of extra whitespace.
        return match.group(1).strip()
    else:
        raise ValueError("No valid Python code block found in the response.")

# ---------------------------
# Function: generate_python_function
# ---------------------------
def generate_python_function(description): 
    response = ollama.chat(model='deepseek-r1:latest', messages=[
        {'role': 'user', 'content': f'Generate a Python function for: {description}'}
    ])   
    return response['message']['content']

# ---------------------------
# Function: sandbox_run
# ---------------------------
def sandbox_run(code: str, env: dict[str, Any]) -> tuple[str, dict[str, Any]]:
    """
    Execute code safely and capture stdout.
    """
    _locals = env.copy()
    original_keys = set(_locals.keys())
    try:
        f = io.StringIO()
        with contextlib.redirect_stdout(f):
            exec(code, {"__builtins__": builtins.__dict__}, _locals)
        result_output = f.getvalue().strip() or "<no output>"
    except Exception as e:
        # Capture and include the full traceback for debugging purposes
        result_output = f"Error during execution: {e}\n{traceback.format_exc()}"
    new_vars = {k: _locals[k] for k in _locals.keys() - original_keys}
    return result_output, new_vars


# ---------------------------
# Main execution to test integration
# ---------------------------
def main():
    """
    Main execution function to generate Python code, extract it, format correctly,
    execute safely, and print outputs.
    """
    # Generate Python function code using Ollama
    description = 'fibonacci sequence generator'
    raw_generated_response = generate_python_function(description)
    print("Raw Generated Response:\n", raw_generated_response)
    
    # Extract only the Python code from the generated response
    try:
        generated_code = extract_python_code(raw_generated_response)
    except ValueError as e:
        print("Error extracting code:", e)
        return

    # Debug: Print the raw representation of the extracted code to inspect whitespace and formatting
    print("\n[DEBUG] Extracted Code Representation:")
    print(repr(generated_code))
    
    # Attempt to compile the generated code to catch syntax errors early
    try:
        compile(generated_code, '<string>', 'exec')
    except Exception as e:
        print("Compilation error in generated code:", e)
        return

    # Prepare the test code without disturbing the generated code's indentation
    test_code = f"""
{generated_code}

# Example usage of generated function
if __name__ == '__main__':
    fib_sequence = fibonacci(10)
    print('Fibonacci sequence:', fib_sequence)
"""
    # Debug: Print the complete test code for verification
    print("\n[DEBUG] Complete Test Code:")
    print(repr(test_code))

    # Execute the test code safely using the sandbox
    output, new_vars = sandbox_run(test_code, {})
    print("\nSandbox Execution Output:\n", output)




if __name__ == "__main__":
    main()


Raw Generated Response:
 <think>
Okay, I need to create a Python function that generates the Fibonacci sequence. Hmm, what's the Fibonacci sequence again? Oh right, it starts with 0 and 1, and each subsequent number is the sum of the previous two. So like 0, 1, 1, 2, 3, 5, etc.

First, I should decide how to make this function. Should it take a specific number of terms as an argument? That makes sense because then the user can specify how long they want the sequence. Let's say the function is called fibonacci and takes n as an input.

What if someone calls the function with n=0 or 1? Well, for n=0, maybe return an empty list. For n=1, return [0]. For higher numbers, start building from there.

So I'll initialize a list to hold the sequence. If n is 0, return an empty list. Else, add 0 and 1 as the first two elements if n is at least that big.

Then, for each subsequent term up to n terms, calculate it by adding the last two numbers. Append each new number to the list until we reach the

In [4]:
import ollama
import builtins
import contextlib
import io
import math
from typing import Any
from langgraph_codeact import create_codeact
from langgraph.checkpoint.memory import MemorySaver

# Existing eval function (unchanged)
def eval(code: str, _locals: dict[str, Any]) -> tuple[str, dict[str, Any]]:
    # Store original keys before execution
    original_keys = set(_locals.keys())
    try:
        with contextlib.redirect_stdout(io.StringIO()) as f:
            exec(code, builtins.__dict__, _locals)
        result = f.getvalue()
        if not result:
            result = "<code ran, no output printed to stdout>"
    except Exception as e:
        result = f"Error during execution: {repr(e)}"
    # Determine new variables created during execution
    new_keys = set(_locals.keys()) - original_keys
    new_vars = {key: _locals[key] for key in new_keys}
    return result, new_vars

def add(a: float, b: float) -> float:
    """Add two numbers together."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers together."""
    return a * b

def divide(a: float, b: float) -> float:
    """Divide one number by another."""
    return a / b

def subtract(a: float, b: float) -> float:
    """Subtract one number from another."""
    return a - b

def sin(a: float) -> float:
    """Calculate the sine of a number (in radians)."""
    return math.sin(a)

def cos(a: float) -> float:
    """Calculate the cosine of a number (in radians)."""
    return math.cos(a)

def radians(a: float) -> float:
    """Convert degrees to radians."""
    return math.radians(a)

def exponentiation(a: float, b: float) -> float:
    """Raise a number to the power of another."""
    return a ** b

def sqrt(a: float) -> float:
    """Compute the square root of a number."""
    return math.sqrt(a)

def ceil(a: float) -> float:
    """Round a number up to the nearest integer."""
    return math.ceil(a)


# List of tool functions
tools = [
    add,
    multiply,
    divide,
    subtract,
    sin,
    cos,
    radians,
    exponentiation,
    sqrt,
    ceil,
]

class OllamaChatModel:
    """
    Wrapper for the Ollama chat model to mimic the interface expected by create_codeact.
    
    This class provides a stream method that accepts input messages,
    calls the Ollama API, and yields output in the expected format.
    """
    def __init__(self, model_name: str):
        self.model_name = model_name
        
    def invoke(self, messages: dict, **kwargs) -> Any:
        """
        Synchronously invoke the model by wrapping the streaming interface.
        This method calls `stream` and returns the first message object.
        """
        for typ, chunk in self.stream(messages, **kwargs):
            if typ == "messages":
                # Return the first message from the chunk
                return chunk[0]
        # Optionally, raise an error if no message was returned.
        raise ValueError("No message returned from stream invocation.")
    
    def stream(self, inputs: dict, stream_mode=None, config=None):
        """
        Mimic streaming behavior for the chat agent.
        
        Parameters:
            inputs: Either a dictionary with a 'messages' key or directly a list of messages.
            stream_mode: (Optional) List of modes to stream.
            config: (Optional) Additional configuration.
                
        Yields:
            A tuple (typ, chunk) where:
            - typ "messages" contains a list with a single message object that has a 'content' attribute.
            - typ "values" yields a dummy dictionary (if needed by the agent).
        """
        # Support both dict input and list input
        if isinstance(inputs, dict):
            messages = inputs.get("messages", [])
        elif isinstance(inputs, list):
            messages = inputs
        else:
            raise TypeError("Invalid input type for stream; expected dict or list.")

        # Call the Ollama API with the provided messages
        response = ollama.chat(model=self.model_name, messages=messages)
        # Create a simple message object with a 'content' attribute
        message_obj = type("Message", (), {})()
        message_obj.content = response['message']['content']
        # Yield the response similar to the expected interface
        yield ("messages", [message_obj])
        # Optionally yield additional values if required by downstream processes
        yield ("values", {"dummy": "value"})


# New agent initialization function using the Ollama model
def initialize_agent_with_ollama():
    """
    Initialize the code_act agent using the OllamaChatModel.
    
    This function replaces the standard init_chat_model call with our Ollama-based implementation.
    It integrates the new chat model into create_codeact and compiles the agent.
    """
    # Instantiate the Ollama-based chat model with the desired model name
    model = OllamaChatModel("deepseek-r1:latest")
    # Create code_act with the new model, our tools list, and the eval function
    code_act = create_codeact(model, tools, eval)
    # Compile the agent using MemorySaver as the checkpoint
    agent = code_act.compile(checkpointer=MemorySaver())
    return agent

if __name__ == "__main__":
    # Initialize the agent using the Ollama model wrapper
    agent = initialize_agent_with_ollama()
    
    messages = [
        {
            "role": "user",
            "content": (
                "A batter hits a baseball at 45.847 m/s at an angle of 23.474° above the horizontal. "
                "The outfielder, who starts facing the batter, picks up the baseball as it lands, then throws it back "
                "towards the batter at 24.12 m/s at an angle of 39.12 degrees. How far is the baseball from where the "
                "batter originally hit it? Assume zero air resistance."
            ),
        }
    ]
    
    # Stream the agent's response and print the output
    for typ, chunk in agent.stream(
        {"messages": messages},
        stream_mode=["values", "messages"],
        config={"configurable": {"thread_id": 1}},
    ):
        if typ == "messages":
            print(chunk[0].content, end="")
        elif typ == "values":
            print("\n\n---answer---\n\n", chunk)





---answer---

 {'messages': [HumanMessage(content='A batter hits a baseball at 45.847 m/s at an angle of 23.474° above the horizontal. The outfielder, who starts facing the batter, picks up the baseball as it lands, then throws it back towards the batter at 24.12 m/s at an angle of 39.12 degrees. How far is the baseball from where the batter originally hit it? Assume zero air resistance.', additional_kwargs={}, response_metadata={}, id='b40d87a4-2e4d-4a3a-b301-e5c4582b29b6')]}


ValidationError: 1 validation error for Message
role
  Field required [type=missing, input_value={'content': 'A batter hit...4a3a-b301-e5c4582b29b6'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing